# **Eclat for Movie Recomendation Analysis**

Menggunakan algoritma Eclat untuk mendapatkan rekomendasi film berdasarkan aktor, sutradara dan genre

Eclat pada dasarnya adalah versi apriori yang disederhanakan, yang hanya mempertimbangkan support, daripada menggunakan ketiganya seperti apriori, yaitu support, Confidence , dan Lift. Oleh karena itu, dapat diimplementasikan dengan cara yang sama seperti apriori, dengan hanya mempertimbangkan Support selama visualisasi

In [ ]:
pip install apyori

  Created wheel for apyori: filename=apyori-1.1.2-cp36-none-any.whl size=5975 sha256=71e7bad2c5ad67fbd0bf63cb50a7198d6ba34e4cdf0b58ec85e4bf09de463187
  Stored in directory: /root/.cache/pip/wheels/5d/92/bb/474bbadbc8c0062b9eb168f69982a0443263f8ab1711a8cad0
Successfully built apyori


In [ ]:
import pandas as pd
import numpy as np
from apyori import apriori
import random

## Preprocessing the data 

In [ ]:
url = 'https://raw.githubusercontent.com/furqoncreative/data-mining-eclat/master/movie_metadata.csv'
dataset=pd.read_csv(url)
dataset.fillna(value=0,axis=1,inplace=True)
dataset.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,0,Doug Walker,0.0,0.0,131.0,0.0,Rob Walker,131.0,0.0,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens,8,143,0,0.0,0,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,0.0,0,0,0,0.0,0.0,12.0,7.1,0.00,0


In [ ]:
dataset.shape

(5043, 28)

Menampung dataset dalam bentuk array

In [ ]:
movieData=[]
for i in range(0,1000):
    movieData.append([str(dataset.values[i,j]) for j in range(0,28)])

In [ ]:
movieData[:1]

[['Color',
  'James Cameron',
  '723.0',
  '178.0',
  '0.0',
  '855.0',
  'Joel David Moore',
  '1000.0',
  '760505847.0',
  'Action|Adventure|Fantasy|Sci-Fi',
  'CCH Pounder',
  'Avatar\xa0',
  '886204',
  '4834',
  'Wes Studi',
  '0.0',
  'avatar|future|marine|native|paraplegic',
  'http://www.imdb.com/title/tt0499549/?ref_=fn_tt_tt_1',
  '3054.0',
  'English',
  'USA',
  'PG-13',
  '237000000.0',
  '2009.0',
  '936.0',
  '7.9',
  '1.78',
  '33000']]

Membuat sampel transaksi kebiasan user menonton film dalam 30 hari.
Dengan asumsi bahwa film yang ditonton selanjutnya:
1. memiliki aktor yang sama dari film sebelumnya, atau
2. memiliki sutradara yang sama dari film sebelumnya, atau
3. memiliki genre yang sama dari film sebelumnya

In [ ]:
def getNextMovie(i,s):
    for i in range(i,1000):
      if (movieData)[i][10] == s[10] or (movieData)[i][9] == s[1] or (movieData)[i][9] == s[9]:
        if movieData[i][11] != s[11]:
          return movieData[i]
        else:
          return random.choice(movieData)
      else: 
        return random.choice(movieData)

In [ ]:
movieTransactions=[]
userTransactions=[]
for i in range(0,1000):
    firtsMovie = (random.choice(movieData))
    userTransactions.append([])
    movieTransactions.append([])
    movieTransactions[i].append(getNextMovie(i,firtsMovie))
    for j in range(0,30):
      movieTransactions[i].append(getNextMovie(i,movieTransactions[i][j]))
      userTransactions[i].append(movieTransactions[i][j][11].replace(u'\xa0', u''))

In [ ]:
userTransactions[:3]

[['Chain Reaction',
  'The Secret Life of Pets',
  'Constantine',
  'Tangled',
  'Kill Bill: Vol. 2',
  'The A-Team',
  'Big Fish',
  'Ted',
  'Hercules',
  'Star Trek',
  'Twister',
  'John Carter',
  'Semi-Pro',
  'The Spiderwick Chronicles',
  'The Girl with the Dragon Tattoo',
  "Something's Gotta Give",
  'Pocahontas',
  'Flight of the Phoenix',
  'Gods of Egypt',
  'Marley & Me',
  'The Jackal',
  '22 Jump Street',
  'The 13th Warrior',
  'Hannibal',
  'Stepmom',
  'Underworld: Awakening',
  'Treasure Planet',
  'Mission: Impossible III',
  'Marmaduke',
  'Open Season'],
 ['Seven Years in Tibet',
  'The Nutty Professor',
  'Interview with the Vampire: The Vampire Chronicles',
  'Ladder 49',
  'Dredd',
  'Hotel Transylvania 2',
  'Flight of the Phoenix',
  'We Are Marshall',
  'The Truman Show',
  'Everest',
  'Elektra',
  'The Black Dahlia',
  'Cinderella Man',
  'Instinct',
  "The Devil's Advocate",
  'Talladega Nights: The Ballad of Ricky Bobby',
  'The Secret Life of Pets',
  

## Training the Eclat model

In [ ]:
# min_support = film ditonton 1x sehari * 30 hari / 1000 user = 0.003
# min_confidence: setidaknya 20%
# min_lift = 2

from apyori import apriori
rules=apriori(userTransactions,min_support=0.003,min_confidence=0.2,min_lift=3,min_length=2,max_length=2)

List hasil data training:

In [ ]:
results=list(rules)
results

[RelationRecord(items=frozenset({'102 Dalmatians', 'San Andreas'}), support=0.005, ordered_statistics=[OrderedStatistic(items_base=frozenset({'102 Dalmatians'}), items_add=frozenset({'San Andreas'}), confidence=0.20833333333333334, lift=8.012820512820513)]),
 RelationRecord(items=frozenset({'Happy Feet 2', '2 Fast 2 Furious'}), support=0.004, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Happy Feet 2'}), items_add=frozenset({'2 Fast 2 Furious'}), confidence=0.2, lift=4.444444444444445)]),
 RelationRecord(items=frozenset({'2 Fast 2 Furious', 'Man on Fire'}), support=0.004, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Man on Fire'}), items_add=frozenset({'2 Fast 2 Furious'}), confidence=0.2, lift=4.444444444444445)]),
 RelationRecord(items=frozenset({'2 Fast 2 Furious', 'Unforgotten'}), support=0.005, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Unforgotten'}), items_add=frozenset({'2 Fast 2 Furious'}), confidence=0.20833333333333334, lift=4.62

## Visualization of Results

In [ ]:
movie,r1,support=[],[],[]
for result in results:
    movie.append(tuple(result[2][0][0])[0])
    r1.append(tuple(result[2][0][1])[0])
    support.append(result[1])

In [ ]:
columns=list(zip(movie,r1,support))

Converting into dataframe

In [ ]:
result_df=pd.DataFrame(columns)
result_df.columns=['Film yang dipilih','Rekomendasi','Support']
result_df

,Film yang dipilih,Rekomendasi,Support
0,102 Dalmatians,San Andreas,0.005
1,Happy Feet 2,2 Fast 2 Furious,0.004
2,Man on Fire,2 Fast 2 Furious,0.004
3,Unforgotten,2 Fast 2 Furious,0.005
4,2 Guns,Ted 2,0.006
...,...,...,...
222,"You, Me and Dupree",The Twilight Saga: Breaking Dawn - Part 2,0.006
223,Thunderbirds,The Wolfman,0.005
224,Titan A.E.,Total Recall,0.006
225,True Lies,Tower Heist,0.006


Diurutkan berdasarkan "Support"

In [ ]:
result_df=result_df.sort_values('Support',ascending=False)
result_df

,Film yang dipilih,Rekomendasi,Support
110,Mr. Popper's Penguins,Fantastic Four,0.008
107,Failure to Launch,Safe House,0.008
156,Last Man Standing,State of Play,0.008
108,Home on the Range,Fantastic Four,0.008
122,Treasure Planet,Godzilla Resurgence,0.007
...,...,...,...
42,Bedtime Stories,The Dark Knight,0.003
41,Bedtime Stories,Talladega Nights: The Ballad of Ricky Bobby,0.003
95,The Golden Compass,Elysium,0.003
39,Bedtime Stories,Superman II,0.003
